# The Forward and the Backward passes

In [0]:
#exports
import sys
sys.path.insert(0,"/content/drive/My Drive/Colab Notebooks/")

from exp.nb_01 import *

In [0]:
def get_data():
  path = datasets.download_data(MNIST_URL, ext= '.gz')
  with gzip.open(path,'rb') as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f,encoding='latin-1')
  return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalise(x, m, s): return (x-m)/s

In [0]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [0]:
x_train = normalise(x_train, train_mean, train_std) #Normalise train data
x_valid = normalise(x_valid, train_mean, train_std) #Normalise valid data using train mean and std

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.0001), tensor(1.))

Note that mean has approached 0 and std 1